УСТАНОВКА НЕОБХОДИМЫХ БИБЛИОТЕК

In [ ]:
%pip install pandas seaborn matplotlib ipywidgets

ИМПОРТ БИБЛИОТЕК

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime  # На всякий случай, если понадобится
import ipywidgets as widgets

# Настраиваем спокойную тему оформления
sns.set_theme()


ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ

In [ ]:
# Параметр порога, ниже которого редкие категории объединяются в "Прочее"
THRESHOLD = 5

# Загрузка датасета
df = pd.read_csv('vgsales-12-4-2019-short.csv')

# Проверка и замена пропусков
df.fillna("неизвестное", inplace=True)

# Приведение столбцов (если нужно, например, Year в целое)
# Допустим, в датасете Year может быть float/строкой, или уже целое. Для надёжности:
if df['Year'].dtype == object:
    # Если есть нечисловые значения, они при ошибке заменятся на NaN => "неизвестное"
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').fillna(-1).astype(int)
else:
    df['Year'] = df['Year'].fillna(-1).astype(int)

# Функция для группировки редких категорий
def group_rare_categories(series, threshold=THRESHOLD):
    """
    Принимает Series (например, Genre, Platform или Publisher).
    Категории, встречающиеся реже threshold раз, переименовывает в "Прочее".
    Возвращает Series с изменёнными категориями.
    """
    value_counts = series.value_counts()
    rare_categories = value_counts[value_counts < threshold].index
    return series.apply(lambda x: "Прочее" if x in rare_categories else x)

# Применяем функцию к выбранным столбцам
df['Genre'] = group_rare_categories(df['Genre'], THRESHOLD)
df['Platform'] = group_rare_categories(df['Platform'], THRESHOLD)
df['Publisher'] = group_rare_categories(df['Publisher'], THRESHOLD)
df['Developer'] = group_rare_categories(df['Developer'], THRESHOLD)

# Для удобства посмотрим, какие столбцы вообще есть
df.info()
df.head()


ФУНКЦИЯ ДЛЯ ПРОВЕРКИ ДОСТУПНОСТИ ЗНАЧЕНИЙ

In [ ]:
def check_existence_in_dataset(col_name: str, value) -> bool:
    """
    Проверяет, есть ли значение value в указанном столбце col_name.
    Если такого значения в данных нет, возвращает False.
    """
    unique_values = df[col_name].unique()
    return value in unique_values


ФУНКЦИЯ ДЛЯ ВЫВОДА/ВИЗУАЛИЗАЦИИ ГЛОБАЛЬНЫХ ПРОДАЖ ПО ГОДУ И ЖАНРУ

In [ ]:
def get_sales_by_year(year: int) -> pd.DataFrame:
    """
    Принимает год year.
    Если год отсутствует в датасете, выводит сообщение
    и возвращает пустой DataFrame.
    В противном случае выводит статистику продаж по жанру 
    и строит (по желанию) визуализации.
    Возвращает отфильтрованный DataFrame.
    """

    if not check_existence_in_dataset('Year', year):
        print("Нет данных по запрошенному параметру, измените вводимые данные на корректные.")
        return pd.DataFrame()  # пустой

    # Фильтрация по году
    df_year = df[df['Year'] == year]

    # Группировка по жанру с суммой глобальных продаж
    sales_by_genre = df_year.groupby('Genre')['Global_Sales'].sum().reset_index()

    # Сортировка
    sales_by_genre.sort_values('Global_Sales', ascending=False, inplace=True)

    print(f"Статистика продаж по жанрам за {year}:")
    display(sales_by_genre)

    return df_year


ФУНКЦИЯ ПОСТРОЕНИЯ КРУГОВОЙ ДИАГРАММЫ ПО ПРОДАЖАМ ЗА УКАЗАННЫЙ ГОД

In [ ]:
def plot_pie_chart_by_year(year: int):
    """
    Строит круговую диаграмму глобальных продаж за указанный год.
    - Радиус долей (процент) отражает 'Global_Sales'.
    - Цвет отражает 'Critic_Score' (от бледного к насыщенному).
    - Если количество жанров > 10, используется легенда, иначе - подписи сегментов.
    - 'Прочее' включается уже в исходном DataFrame.
    """

    # Получаем отфильтрованный DF
    df_year = get_sales_by_year(year)
    if df_year.empty:
        return  # уже вывели сообщение об ошибке

    # Группируем по жанру
    sales_genre = df_year.groupby('Genre', as_index=False).agg({
        'Global_Sales': 'sum',
        'Critic_Score': 'mean'  # для цвета возьмём средний Critic_Score
    })

    # При построении цветовой карты будем использовать Critic_Score
    # Чем выше Critic_Score, тем насыщеннее цвет.

    # Сначала normalize Critic_Score (чтобы получить значения от 0 до 1)
    # Если Critic_Score == "неизвестное", при mean это могла быть строка, 
    # но по идее в датафрейме изначально должны быть числа или "неизвестное". 
    # С учётом fillna("неизвестное") придётся исключить такие строки/преобразовать.
    numeric_mask = pd.to_numeric(sales_genre['Critic_Score'], errors='coerce').notna()
    # Чтобы не терять все строки, заменим "неизвестное" на 0
    sales_genre.loc[~numeric_mask, 'Critic_Score'] = 0

    # Теперь преобразуем в float
    sales_genre['Critic_Score'] = sales_genre['Critic_Score'].astype(float)

    min_score = sales_genre['Critic_Score'].min()
    max_score = sales_genre['Critic_Score'].max()
    score_range = max_score - min_score if max_score != min_score else 1

    normalized_colors = (sales_genre['Critic_Score'] - min_score) / score_range

    # Создадим колormap
    cmap = sns.light_palette("blue", as_cmap=True)

    # Генерируем список цветов на основе нормированных значений Critic_Score
    colors = [cmap(x) for x in normalized_colors]

    # Построение круговой диаграммы
    fig, ax = plt.subplots(figsize=(8, 6))
    total_sales = sales_genre['Global_Sales'].sum()

    # Pie chart
    if len(sales_genre) > 10:
        # Без подписей, но с легендой
        wedges, _, autotexts = ax.pie(
            sales_genre['Global_Sales'],
            colors=colors,
            startangle=140,
            autopct=lambda pct: f"{pct:.1f}%"  # процент от общего
        )
        ax.legend(wedges, sales_genre['Genre'], title="Жанр", bbox_to_anchor=(1, 0.5), loc='center left')
    else:
        # Подписи прямо на сегментах
        wedges, texts, autotexts = ax.pie(
            sales_genre['Global_Sales'],
            labels=sales_genre['Genre'],
            colors=colors,
            startangle=140,
            autopct=lambda pct: f"{pct:.1f}%" 
        )

    ax.set_title(f"Круговая диаграмма по жанрам за {year}\n(Общие продажи: {total_sales:.2f} млн)")
    plt.show()


ПЕРВАЯ ФУНКЦИЯ ДЛЯ SCATTER-ПЛОТА (ЦВЕТ = GENRE)

In [ ]:
def scatter_plot_genre_color():
    """
    Строит scatter plot:
    - x = Critic_Score
    - y = User_Score
    - цвет (hue) = Genre
    - размер (size) = Global_Sales
    Возвращает исходный DataFrame (можно дальше фильтровать).
    """

    # Уберём строки, где Critic_Score или User_Score = "неизвестное"
    df_scatter = df.copy()
    df_scatter = df_scatter[df_scatter['Critic_Score'] != "неизвестное"]
    df_scatter = df_scatter[df_scatter['User_Score'] != "неизвестное"]

    # Преобразуем в float
    df_scatter['Critic_Score'] = df_scatter['Critic_Score'].astype(float)
    df_scatter['User_Score'] = df_scatter['User_Score'].astype(float)

    # Построение
    plt.figure(figsize=(10, 6))
    scatter = sns.scatterplot(
        data=df_scatter,
        x='Critic_Score',
        y='User_Score',
        hue='Genre',
        size='Global_Sales',
        alpha=0.7,
        palette='husl'
    )
    scatter.set_title("Соотношение Critic_Score и User_Score (цвет = Жанр, размер = Global_Sales)")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
    plt.show()

    return df_scatter


ВТОРАЯ ФУНКЦИЯ ДЛЯ SCATTER-ПЛОТА (ЦВЕТ = PLATFORM)

In [ ]:
def scatter_plot_platform_color():
    """
    Строит scatter plot:
    - x = Critic_Score
    - y = User_Score
    - цвет (hue) = Platform (или Publisher, по желанию)
    - размер (size) = Global_Sales
    Возвращает исходный DataFrame (можно дальше фильтровать).
    """

    # Уберём строки, где Critic_Score или User_Score = "неизвестное"
    df_scatter = df.copy()
    df_scatter = df_scatter[df_scatter['Critic_Score'] != "неизвестное"]
    df_scatter = df_scatter[df_scatter['User_Score'] != "неизвестное"]

    # Преобразуем в float
    df_scatter['Critic_Score'] = df_scatter['Critic_Score'].astype(float)
    df_scatter['User_Score'] = df_scatter['User_Score'].astype(float)

    # Построение
    plt.figure(figsize=(10, 6))
    scatter = sns.scatterplot(
        data=df_scatter,
        x='Critic_Score',
        y='User_Score',
        hue='Platform',
        size='Global_Sales',
        alpha=0.7,
        palette='husl'
    )
    scatter.set_title("Соотношение Critic_Score и User_Score (цвет = Платформа, размер = Global_Sales)")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
    plt.show()

    return df_scatter


ФУНКЦИЯ ДЛЯ СВОДНОЙ ТАБЛИЦЫ (PIVOT)

In [ ]:
def create_pivot_table(rows=('Year', 'Genre'), 
                       cols=('Platform',), 
                       values='Global_Sales', 
                       aggfunc='sum') -> pd.DataFrame:
    """
    Создаёт сводную таблицу по выбранным параметрам.
    rows, cols - кортежи колонок, которые будут 
                 использоваться в качестве строк и столбцов.
    values - поле для агрегации (по умолчанию Global_Sales).
    aggfunc - функция агрегации (по умолчанию sum).

    Возвращает pd.DataFrame (pivot).
    """

    pivot_df = pd.pivot_table(
        df,
        index=list(rows),
        columns=list(cols),
        values=values,
        aggfunc=aggfunc,
        fill_value=0
    )

    print(f"Сводная таблица по параметрам: rows={rows}, cols={cols}, values={values}, aggfunc={aggfunc}")
    display(pivot_df)

    return pivot_df


ФУНКЦИЯ ВЫВОДА СТАТИСТИЧЕСКИХ МЕТРИК

In [ ]:
def print_statistics(filtered_df: pd.DataFrame):
    """
    Принимает DataFrame (уже отфильтрованный).
    Выводит среднее, медиану, макс, мин для Global_Sales, Critic_Score, User_Score.
    """

    # Обработка "неизвестное"
    df_stats = filtered_df.copy()
    for col in ['Critic_Score', 'User_Score', 'Global_Sales']:
        # Заменяем "неизвестное" на NaN
        df_stats[col] = pd.to_numeric(df_stats[col], errors='coerce')

    metrics = {}
    for col in ['Global_Sales', 'Critic_Score', 'User_Score']:
        valid_values = df_stats[col].dropna()
        if len(valid_values) == 0:
            metrics[col] = {
                'mean': None,
                'median': None,
                'max': None,
                'min': None
            }
        else:
            metrics[col] = {
                'mean': valid_values.mean(),
                'median': valid_values.median(),
                'max': valid_values.max(),
                'min': valid_values.min()
            }

    print("Статистические метрики (Global_Sales, Critic_Score, User_Score):")
    for col, vals in metrics.items():
        print(f"--- {col} ---")
        print(f"  Среднее: {vals['mean']}")
        print(f"  Медиана: {vals['median']}")
        print(f"  Макс:    {vals['max']}")
        print(f"  Мин:     {vals['min']}")


ФУНКЦИЯ ДЛЯ ПОСТРОЕНИЯ НЕБОЛЬШОГО BAR-ЧАРТА (ПРОГРЕСС-БАР) ПО ФИЛЬТРАМ

In [ ]:
def plot_global_sales_bar(filtered_df: pd.DataFrame):
    """
    Принимает уже отфильтрованный DataFrame.
    Строит bar chart (одна колонка), показывающий суммарные продажи (Global_Sales).
    """
    # Преобразуем Global_Sales в float
    temp_df = filtered_df.copy()
    temp_df['Global_Sales'] = pd.to_numeric(temp_df['Global_Sales'], errors='coerce').fillna(0)

    total_sales = temp_df['Global_Sales'].sum()

    fig, ax = plt.subplots(figsize=(4, 3))
    ax.barh(['Суммарные продажи'], [total_sales], color='gray')
    ax.set_title("Суммарные глобальные продажи по фильтру")
    ax.set_xlabel("Global Sales (млн)")
    for i, v in enumerate([total_sales]):
        ax.text(v + 0.1, i, f"{v:.2f}", va='center')
    plt.show()


КНОПКА ДЛЯ СОХРАНЕНИЯ ТЕКУЩЕГО ГРАФИКА

In [ ]:
save_button = widgets.Button(description="Сохранить график в PNG")

def save_current_figure(btn):
    """
    Коллбэк для кнопки save_button, сохраняет текущую фигуру в файл .png.
    """
    # Можно придумать название файла, например, исходя из текущей даты/времени
    import time
    filename = f"figure_{time.strftime('%Y%m%d_%H%M%S')}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"График сохранён в файл {filename}")

save_button.on_click(save_current_figure)

display(save_button)


ТЕПЛОВАЯ КАРТА (HEATMAP) КОРРЕЛЯЦИИ

In [ ]:
def plot_correlation_heatmap():
    """
    Строит тепловую карту корреляции только между Critic_Score, User_Score, Global_Sales.
    Annot=True, fmt=".2f".
    Обязательный заголовок, спокойный стиль.
    """

    # Выберем нужные столбцы
    corr_df = df[['Critic_Score', 'User_Score', 'Global_Sales']].copy()

    # Заменим "неизвестное" на NaN
    for col in corr_df.columns:
        corr_df[col] = pd.to_numeric(corr_df[col], errors='coerce')

    # Считаем корреляцию
    corr_matrix = corr_df.corr()

    plt.figure(figsize=(5, 4))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='Blues', vmin=-1, vmax=1)
    plt.title("Тепловая карта корреляций (Critic_Score, User_Score, Global_Sales)")
    plt.show()


ИНТЕРАКТИВНЫЕ ВИДЖЕТЫ ДЛЯ ФИЛЬТРАЦИИ
Пример, как можно сделать базовый набор фильтров:
  Dropdown по году, жанру, платформе; MultipleSelect по издателю/разработчику и т.д.

In [ ]:


# Получаем уникальные значения
unique_years = sorted([y for y in df['Year'].unique() if y != -1])  # -1 мы обозначили "неизвестное"
unique_genres = sorted(df['Genre'].unique())
unique_platforms = sorted(df['Platform'].unique())
unique_publishers = sorted(df['Publisher'].unique())
unique_devs = sorted(df['Developer'].unique())

year_dropdown = widgets.Dropdown(
    options=['Все'] + unique_years,
    description='Год:',
    value='Все'
)

genre_dropdown = widgets.Dropdown(
    options=['Все'] + unique_genres,
    description='Жанр:',
    value='Все'
)

platform_dropdown = widgets.Dropdown(
    options=['Все'] + unique_platforms,
    description='Платформа:',
    value='Все'
)

publisher_select = widgets.SelectMultiple(
    options=['Все'] + unique_publishers,
    description='Издатели:',
    value=['Все']
)

dev_select = widgets.SelectMultiple(
    options=['Все'] + unique_devs,
    description='Разработчики:',
    value=['Все']
)

def filter_data(year, genre, platform, publishers, devs) -> pd.DataFrame:
    """
    Фильтрует df по выбранным параметрам (если 'Все', то игнорируем).
    Возвращает отфильтрованный DataFrame.
    """
    filtered = df.copy()

    # Год
    if year != 'Все':
        if not check_existence_in_dataset('Year', year):
            print("Нет данных по запрошенному году, измените вводимые данные на корректные.")
            return pd.DataFrame()  # Пустой
        filtered = filtered[filtered['Year'] == year]

    # Жанр
    if genre != 'Все':
        if not check_existence_in_dataset('Genre', genre):
            print("Нет данных по запрошенному жанру, измените вводимые данные на корректные.")
            return pd.DataFrame()
        filtered = filtered[filtered['Genre'] == genre]

    # Платформа
    if platform != 'Все':
        if not check_existence_in_dataset('Platform', platform):
            print("Нет данных по запрошенной платформе, измените вводимые данные на корректные.")
            return pd.DataFrame()
        filtered = filtered[filtered['Platform'] == platform]

    # Издатели
    if publishers != ['Все']:
        # Проверим, все ли издатели валидны
        for pub in publishers:
            if pub != 'Все' and not check_existence_in_dataset('Publisher', pub):
                print("Нет данных по запрошенному(ым) издателю(ям), измените вводимые данные на корректные.")
                return pd.DataFrame()
        # Фильтруем, если в списке издателей
        # Если выбрано несколько, оставляем записи, где Publisher входит в publishers
        valid_pubs = [p for p in publishers if p != 'Все']
        if valid_pubs:
            filtered = filtered[filtered['Publisher'].isin(valid_pubs)]

    # Разработчики
    if devs != ['Все']:
        for d in devs:
            if d != 'Все' and not check_existence_in_dataset('Developer', d):
                print("Нет данных по запрошенному(ым) разработчику(ам), измените вводимые данные на корректные.")
                return pd.DataFrame()
        valid_devs = [d for d in devs if d != 'Все']
        if valid_devs:
            filtered = filtered[filtered['Developer'].isin(valid_devs)]

    return filtered


def on_change_filter(*args):
    # Получаем значения виджетов
    year_val = year_dropdown.value
    genre_val = genre_dropdown.value
    platform_val = platform_dropdown.value
    publisher_val = list(publisher_select.value)
    dev_val = list(dev_select.value)

    filtered_df = filter_data(year_val, genre_val, platform_val, publisher_val, dev_val)

    if filtered_df.empty:
        print("Отфильтрованный DataFrame пуст. Проверьте корректность введённых фильтров.")
    else:
        # Выводим небольшое резюме
        print(f"Количество строк в отфильтрованном DataFrame: {len(filtered_df)}")

        # Можно сразу отобразить статистику и мини-график
        print_statistics(filtered_df)
        plot_global_sales_bar(filtered_df)

# Подключаем обработчик к каждому виджету
year_dropdown.observe(on_change_filter, 'value')
genre_dropdown.observe(on_change_filter, 'value')
platform_dropdown.observe(on_change_filter, 'value')
publisher_select.observe(on_change_filter, 'value')
dev_select.observe(on_change_filter, 'value')

display(year_dropdown, genre_dropdown, platform_dropdown, publisher_select, dev_select)


# Game Sales Analysis

## Introduction
This project, **Game Sales Analysis**, focuses on analyzing video game sales data up to 2019. The goal is to explore, visualize, and understand trends in the gaming industry using Python.

### Why is this project important?
The project is useful for understanding:
- Market dynamics in the gaming industry.
- Popular game genres and platforms over time.
- Patterns in global sales distribution.

### Authors
[Your Name]

## Key Features
- **Data Manipulation**: Add, delete, and view records using pandas and NumPy.
- **Data Visualization**: Generate histograms, bar charts, and pie charts with matplotlib.
- **Filtering and Grouping**: Filter games by year and group data for analysis.
- **Example Code**: Includes practical examples for data preprocessing and visualization.

## Project Structure
```
project-folder/
|
|-- LICENSE
|-- README.md
|-- requirements.txt
|-- vgsales-12-4-2019-short.csv
|-- vgsales.ipynb
```

## Installation
1. Install **Python 3.11.9** or later.
2. Clone the repository or download the files.
3. Install dependencies:
   ```bash
   pip install -r requirements.txt
   ```

## Examples of Use

### Data Preprocessing
```python
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('vgsales-12-4-2019-short.csv')

# Fill missing values
df.fillna("unknown", inplace=True)

# Convert 'Year' to integer
df['Year'] = pd.to_numeric(df['Year'], errors='coerce').fillna(-1).astype(int)

# Drop duplicates
df.drop_duplicates(inplace=True)

print(df.info())
```

### Visualizations

#### Histogram
```python
import matplotlib.pyplot as plt

# Global Sales Histogram
df['Global_Sales'].hist(bins=20)
plt.title("Distribution of Global Sales")
plt.xlabel("Sales (Millions)")
plt.ylabel("Frequency")
plt.show()
```

#### Bar Chart
```python
# Sales by Genre
genre_sales = df.groupby('Genre')['Global_Sales'].sum()
genre_sales.sort_values().plot(kind='bar', figsize=(10, 6))
plt.title("Sales by Genre")
plt.xlabel("Genre")
plt.ylabel("Global Sales (Millions)")
plt.show()
```

#### Pie Chart
```python
# Platform distribution
platform_sales = df['Platform'].value_counts()
platform_sales.plot(kind='pie', autopct='%1.1f%%', figsize=(8, 8))
plt.title("Platform Distribution")
plt.ylabel("")  # Hides the y-label
plt.show()
```

## Technical Requirements
- **Python**: Version 3.11.9
- **Dependencies**:
  - pandas (>= 1.5.3)
  - numpy (>= 1.24.3)
  - matplotlib (>= 3.7.1)

## License
This project is licensed under the MIT License. See the [LICENSE](./LICENSE) file for details.

## Contact
For inquiries, please contact:
**Email**: your.email@example.com

## References
- [pandas Documentation](https://pandas.pydata.org/docs/)
- [NumPy Documentation](https://numpy.org/doc/)
- [matplotlib Documentation](https://matplotlib.org/stable/contents.html)
- [Dataset on Kaggle](https://www.kaggle.com/)

